In [1]:
import sys
import os
import glob
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx 
%matplotlib inline

In [40]:
df = pd.read_csv('../../01 Data/03 Consolidated/authors_with_gender_v1.csv')

In [41]:
df

,paper_id,date,name,fname,lname,affiliationIds,norm_fname,gender
0,10.1103/PhysRevApplied.9.039901,2018-03-06,Tomoyuki Sasaki,Tomoyuki,Sasaki,[],tomoyuki,NaN
1,10.1103/PhysRevApplied.9.039901,2018-03-06,Yuichiro Ando,Yuichiro,Ando,[],yuichiro,NaN
2,10.1103/PhysRevApplied.9.039901,2018-03-06,Makoto Kameno,Makoto,Kameno,[],makoto,male
3,10.1103/PhysRevApplied.9.039901,2018-03-06,Takayuki Tahara,Takayuki,Tahara,[],takayuki,male
4,10.1103/PhysRevApplied.9.039901,2018-03-06,Hayato Koike,Hayato,Koike,[],hayato,NaN
...,...,...,...,...,...,...,...,...
2503838,10.1103/PhysRevSTAB.14.014001,2011-01-21,Mikhail Zobov,Mikhail,Zobov,a2,mikhail,male
2503839,10.1103/PhysRevSTAB.14.090401,2011-09-15,A. Lorusso,A.,Lorusso,a1,a,NaN
2503840,10.1103/PhysRevSTAB.14.090401,2011-09-15,F. Gontad,F.,Gontad,a1,f,NaN
2503841,10.1103/PhysRevSTAB.14.090401,2011-09-15,A. Perrone,A.,Perrone,a1,a,NaN


In [42]:
# Change format to datetime
df['date'] = df['date'].astype('datetime64[ns]')

# As no time val, extract date, yrmo, yr
df['YearMonth'] = df['date'].map(lambda x: 100*x.year + x.month)
df['year'] = pd.DatetimeIndex(df['date']).year

In [43]:
df.gender.fillna('not_found', inplace=True)

In [44]:
df = df.drop_duplicates(['paper_id','name','gender','year'], keep='first')

In [45]:
df

,paper_id,date,name,fname,lname,affiliationIds,norm_fname,gender,YearMonth,year
0,10.1103/PhysRevApplied.9.039901,2018-03-06,Tomoyuki Sasaki,Tomoyuki,Sasaki,[],tomoyuki,not_found,201803,2018
1,10.1103/PhysRevApplied.9.039901,2018-03-06,Yuichiro Ando,Yuichiro,Ando,[],yuichiro,not_found,201803,2018
2,10.1103/PhysRevApplied.9.039901,2018-03-06,Makoto Kameno,Makoto,Kameno,[],makoto,male,201803,2018
3,10.1103/PhysRevApplied.9.039901,2018-03-06,Takayuki Tahara,Takayuki,Tahara,[],takayuki,male,201803,2018
4,10.1103/PhysRevApplied.9.039901,2018-03-06,Hayato Koike,Hayato,Koike,[],hayato,not_found,201803,2018
...,...,...,...,...,...,...,...,...,...,...
2503838,10.1103/PhysRevSTAB.14.014001,2011-01-21,Mikhail Zobov,Mikhail,Zobov,a2,mikhail,male,201101,2011
2503839,10.1103/PhysRevSTAB.14.090401,2011-09-15,A. Lorusso,A.,Lorusso,a1,a,not_found,201109,2011
2503840,10.1103/PhysRevSTAB.14.090401,2011-09-15,F. Gontad,F.,Gontad,a1,f,not_found,201109,2011
2503841,10.1103/PhysRevSTAB.14.090401,2011-09-15,A. Perrone,A.,Perrone,a1,a,not_found,201109,2011


In [46]:
df

,paper_id,date,name,fname,lname,affiliationIds,norm_fname,gender,YearMonth,year
0,10.1103/PhysRevApplied.9.039901,2018-03-06,Tomoyuki Sasaki,Tomoyuki,Sasaki,[],tomoyuki,not_found,201803,2018
1,10.1103/PhysRevApplied.9.039901,2018-03-06,Yuichiro Ando,Yuichiro,Ando,[],yuichiro,not_found,201803,2018
2,10.1103/PhysRevApplied.9.039901,2018-03-06,Makoto Kameno,Makoto,Kameno,[],makoto,male,201803,2018
3,10.1103/PhysRevApplied.9.039901,2018-03-06,Takayuki Tahara,Takayuki,Tahara,[],takayuki,male,201803,2018
4,10.1103/PhysRevApplied.9.039901,2018-03-06,Hayato Koike,Hayato,Koike,[],hayato,not_found,201803,2018
...,...,...,...,...,...,...,...,...,...,...
2503838,10.1103/PhysRevSTAB.14.014001,2011-01-21,Mikhail Zobov,Mikhail,Zobov,a2,mikhail,male,201101,2011
2503839,10.1103/PhysRevSTAB.14.090401,2011-09-15,A. Lorusso,A.,Lorusso,a1,a,not_found,201109,2011
2503840,10.1103/PhysRevSTAB.14.090401,2011-09-15,F. Gontad,F.,Gontad,a1,f,not_found,201109,2011
2503841,10.1103/PhysRevSTAB.14.090401,2011-09-15,A. Perrone,A.,Perrone,a1,a,not_found,201109,2011


In [57]:
df[df.name == 'Abbas Ali Saberi']

,paper_id,date,name,fname,lname,affiliationIds,norm_fname,gender,YearMonth,year
1387521,10.1103/PhysRevE.84.021113,2011-08-05,Abbas Ali Saberi,Abbas Ali,Saberi,a1,abbas ali,not_found,201108,2011
1691157,10.1103/PhysRevLett.110.178501,2013-04-24,Abbas Ali Saberi,Abbas,Ali Saberi,a1,abbas,male,201304,2013


In [47]:
df_age = df.groupby(['name','gender'],as_index=False).agg({'paper_id':'count', 'year':[np.min,np.max]})

In [48]:
df_age

name     gender paper_id  year      
                                              count  amin  amax
0        \nDUET Collaboration\n  not_found        1  2017  2017
1       \nSπRIT Collaboration\n  not_found        1  2017  2017
2                       Arianto  not_found        1  2011  2011
3                     Riazuddin  not_found        1  2011  2011
4               A. M. Balbashov  not_found        1  2004  2004
...                         ...        ...      ...   ...   ...
449378       χQCD Collaboration  not_found       14  2009  2018
449379            А. N. Lodygin  not_found        3  2016  2018
449380            О. S. Vaulina  not_found        1  2015  2015
449381                  Ṣ. Uǧur  not_found        1  2015  2015
449382           Å. M. Lindberg  not_found        1  1997  1997

[449383 rows x 5 columns]

In [49]:
df_age.gender.value_counts()

not_found    361193
male          76868
female        11322
Name: gender, dtype: int64

In [50]:
df_age['age'] = df_age['year']['amax'] - df_age['year']['amin'] + 1

In [51]:
df_age

name     gender paper_id  year       age
                                              count  amin  amax    
0        \nDUET Collaboration\n  not_found        1  2017  2017   1
1       \nSπRIT Collaboration\n  not_found        1  2017  2017   1
2                       Arianto  not_found        1  2011  2011   1
3                     Riazuddin  not_found        1  2011  2011   1
4               A. M. Balbashov  not_found        1  2004  2004   1
...                         ...        ...      ...   ...   ...  ..
449378       χQCD Collaboration  not_found       14  2009  2018  10
449379            А. N. Lodygin  not_found        3  2016  2018   3
449380            О. S. Vaulina  not_found        1  2015  2015   1
449381                  Ṣ. Uǧur  not_found        1  2015  2015   1
449382           Å. M. Lindberg  not_found        1  1997  1997   1

[449383 rows x 6 columns]

In [52]:
df2 = pd.DataFrame()
df2['name'] = df_age['name']
df2['gender'] = df_age['gender']
df2['paper_count'] = df_age['paper_id']['count']
df2['age'] = df_age['age']

In [53]:
df2

,name,gender,paper_count,age
0,\nDUET Collaboration\n,not_found,1,1
1,\nSπRIT Collaboration\n,not_found,1,1
2,Arianto,not_found,1,1
3,Riazuddin,not_found,1,1
4,A. M. Balbashov,not_found,1,1
...,...,...,...,...
449378,χQCD Collaboration,not_found,14,10
449379,А. N. Lodygin,not_found,3,3
449380,О. S. Vaulina,not_found,1,1
449381,Ṣ. Uǧur,not_found,1,1


In [54]:
print(np.mean(df2[df2.gender=='male']['age']))
print(np.mean(df2[df2.gender=='female']['age']))

5.48114950304418
4.18477300830242


In [55]:
print(np.mean(df2[df2.gender=='male']['paper_count']))
print(np.mean(df2[df2.gender=='female']['paper_count']))

4.340948118853099
3.4428546193252076


In [56]:
df2.to_csv('../../01 Data/03 Consolidated/author_wo_gender_pprcount_age_v1.csv', index=False)